# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

Import packages, classifiers and etc.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

from imblearn.over_sampling import SMOTE, RandomOverSampler

Import VAEOversampler.

In [2]:
from VAEOversampler import VAEOversampler

2023-07-10 22:17:26.388763: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-10 22:17:26.388798: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [3]:
from imblearn.datasets import fetch_datasets

dset_name = 'libras_move'
dset = fetch_datasets()[dset_name]

X, y = StandardScaler().fit_transform(dset.data), dset.target

In [4]:
X

array([[ 1.15937304, -0.92951044,  1.16413245, ..., -0.82149815,
         0.64799276, -0.8909614 ],
       [ 0.56538106, -1.51057836,  0.57805964, ...,  1.24028011,
        -1.4220029 ,  1.1983437 ],
       [ 0.77958869, -1.70841598,  0.79292607, ..., -1.0729544 ,
         1.28357889, -0.97597143],
       ...,
       [ 0.22456186,  1.08582686,  0.22644625, ...,  0.30994632,
        -0.21095664,  0.27514529],
       [ 0.46796818,  1.25893478,  0.46085517, ...,  0.33509195,
         0.21854026,  0.29944137],
       [ 0.37060565,  0.87565108,  0.3827357 , ...,  0.17167254,
         0.12404384,  0.15371739]])

In [5]:
y[y == -1] = 0

We split data into train and test partitions.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [7]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [8]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))


## Without resampling (base line) 
Which is starting score?


In [9]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 336, 1: 24})
Ratio-> 14.0 : 1


In [10]:
train_val(X_train, y_train, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  LGBMClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  XGBClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  BaggingClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  RandomForestClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Ensemble predictions (majority voting):
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746


## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [11]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f67cc15f430>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [12]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 269, 1: 269})
Ratio->  1 : 1.0


In [13]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  LGBMClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  XGBClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  BaggingClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  RandomForestClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Ensemble predictions (majority voting):
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443


In [14]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [15]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 269, 1: 269})
Ratio->  1 : 1.0


In [16]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  LGBMClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  XGBClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  BaggingClassifier
[[65  2]
 [ 2  3]]

TNR: 0.97015
TPR: 0.6
TNRxTPR: 0.58209
G-mean: 0.76295
__________________________________________________
Classifier:  RandomForestClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Ensemble predictions (majority voting):
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746


In [17]:
# VAEOversampler

vae_sampler = VAEOversampler(epochs=500,
                              intermediate_dim=512,
                              batch_size=64,
                              random_state=42,
                              verbose=False)
X_res, y_res = vae_sampler.fit_resample(X_train, y_train)

2023-07-10 22:18:56.202907: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-10 22:18:56.202953: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-10 22:18:56.203018: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a-Modern-15-A5M): /proc/driver/nvidia/version does not exist
2023-07-10 22:18:56.204359: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0.0: 269, 1.0: 269})
Ratio->  1 : 1.0


In [19]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  LGBMClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  XGBClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  BaggingClassifier
[[66  1]
 [ 2  3]]

TNR: 0.98507
TPR: 0.6
TNRxTPR: 0.59104
G-mean: 0.76879
__________________________________________________
Classifier:  RandomForestClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Ensemble predictions (majority voting):
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746


## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [20]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f67c8352b80>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [25]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.7)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 269, 0: 188})
Ratio->  1 : 1.4


In [26]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  LGBMClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  XGBClassifier
[[66  1]
 [ 1  4]]

TNR: 0.98507
TPR: 0.8
TNRxTPR: 0.78806
G-mean: 0.88773
__________________________________________________
Classifier:  BaggingClassifier
[[62  5]
 [ 2  3]]

TNR: 0.92537
TPR: 0.6
TNRxTPR: 0.55522
G-mean: 0.74513
__________________________________________________
Classifier:  RandomForestClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Ensemble predictions (majority voting):
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443


In [27]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [30]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.6)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 269, 0: 161})
Ratio->  1 : 1.7


In [31]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  LGBMClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  XGBClassifier
[[66  1]
 [ 2  3]]

TNR: 0.98507
TPR: 0.6
TNRxTPR: 0.59104
G-mean: 0.76879
__________________________________________________
Classifier:  BaggingClassifier
[[61  6]
 [ 2  3]]

TNR: 0.91045
TPR: 0.6
TNRxTPR: 0.54627
G-mean: 0.7391
__________________________________________________
Classifier:  RandomForestClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Ensemble predictions (majority voting):
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746


In [50]:
# VAEOversampler

X_res, y_res = vae_sampler.resample(X_train, y_train, sampling_strategy=1)

In [53]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.7)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 269, 0.0: 188})
Ratio->  1 : 1.4


In [54]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[66  1]
 [ 1  4]]

TNR: 0.98507
TPR: 0.8
TNRxTPR: 0.78806
G-mean: 0.88773
__________________________________________________
Classifier:  LGBMClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  XGBClassifier
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  BaggingClassifier
[[65  2]
 [ 2  3]]

TNR: 0.97015
TPR: 0.6
TNRxTPR: 0.58209
G-mean: 0.76295
__________________________________________________
Classifier:  RandomForestClassifier
[[67  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Ensemble predictions (majority voting):
[[67  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443


## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
